# 從dcard感情版中抓取留言的性別比例




In [0]:
!pip install requests
!pip install dcard-spider
!pip install python-dateutil
!git clone https://github.com/leVirve/dcard-spider
from dcard import Dcard
import datetime
from collections import defaultdict

fatal: destination path 'dcard-spider' already exists and is not an empty directory.


In [0]:
from __future__ import unicode_literals, absolute_import
import logging

from dcard.api import api
from dcard.posts import Post
from dcard.utils import flatten_lists
class Forum:
    metas_per_page = 30
    infinite_page = -1

    def __init__(self, name=None):
        self.name = name
        self.api = api

    def __call__(self, name):
        self.name = name
        return self

    def get_metas(
            self, num=30, sort='new', before=None, timebound=None,
            callback=None):
        logger.info('<%s> 開始取得看板內文章資訊', self.name)

        paged_metas = self.api.get_metas(
            self.name, sort, num, before, timebound)

        buff = flatten_lists(metas for metas in paged_metas)
        results = callback(buff) if callback else buff

        logger.info('<%s> 資訊蒐集完成，共%d筆', self.name, len(buff))
        return results

    def get(self, no_school=False):
        if no_school:
            return self.api.get_general_forums()
        return self.api.get_all_forums()


In [0]:
import datetime
from collections import defaultdict
from dcard import Dcard
dcard = Dcard()
analyzed_result = defaultdict(lambda: 0)
target_date = datetime.datetime.utcnow() - datetime.timedelta(days=1) 
def filter_hot(metas):
    return [meta for meta in metas if meta['likeCount'] >= 100]
metas = dcard.forums('relationship').get_metas(
            num=dcard.forums.infinite_page,  
            timebound=target_date.isoformat(),  
            callback=filter_hot  
        )
print('Collect %d metas' % len(metas))
articles = dcard.posts(metas).get(content=False, links=False)
# 分析
for article in articles.result():
    for comment in article['comments']:
        analyzed_result[comment.get('gender')] += 1
print(analyzed_result)

Collect 14 metas
defaultdict(<function <lambda> at 0x7fec0e7b3d90>, {'M': 560, 'F': 713, None: 29, 'D': 7})
